# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
## Load Cities.csv as DataFrame
cities_df = pd.read_csv('Output/Cities.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
## Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
locations = cities_df[['Lat', 'Lng']].values.tolist()
type(locations[0][0])

float

In [5]:
max_humidity = cities_df['Humidity'].max()
max_humidity

100

In [6]:
fig = gmaps.figure(center=(15,15), zoom_level=2.1)

locations = cities_df[['Lat', 'Lng']].values.tolist()
weights = cities_df['Humidity'].values.tolist()
max_humidity = cities_df['Humidity'].max()

heat_layer = gmaps.heatmap_layer(
    locations=locations,
    weights=weights,
    dissipating=False,
    max_intensity=max_humidity,
    point_radius = 3
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
len(cities_df)

592

In [8]:
ideal_conditions = (cities_df['Max Temp'] > 70) & \
                   (cities_df['Max Temp'] < 80) & \
                   (cities_df['Wind Speed'] < 10) & \
                   (cities_df['Cloudiness'] == 0)

In [9]:
sum(ideal_conditions)

8

In [10]:
ideal_cities_df = cities_df[['City', 'Country', 'Lat', 'Lng']].loc[ideal_conditions]
ideal_cities_df.reset_index(drop=True, inplace=True)
ideal_cities_df.dropna(axis=0, inplace=True)

In [11]:
ideal_cities_df

,City,Country,Lat,Lng
0,Samandağ,TR,36.0842,35.9771
1,Yining,CN,29.0361,114.5636
2,Giyani,ZA,-23.3025,30.7187
3,Qurayyat,SA,31.3318,37.3428
4,Jingdezhen,CN,29.2947,117.2079
5,Broken Hill,AU,-31.9500,141.4333
6,Antakya,TR,36.2066,36.1572
7,Yicheng,CN,31.3705,119.8703


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
## GOOGLE MAPS API

## Nearby Search URL
nearby_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

## Nearby Search Parameters
params = {
    'key': g_key,
    'radius': 5000,
    'type': "lodging",
    'language': "en"
}

In [13]:
hotel_name = []
hotel_lat = []
hotel_lng = []

## Print Log Header
print("Beginning Data Retrieval")
print("------------------------------")

for index, row in ideal_cities_df.iterrows():
    params['location'] = f"{row['Lat']},{row['Lng']}"
    response = requests.get(nearby_url, params=params).json()
    first_result = response['results'][0]
    
    print(f"Processing {row['City']}:")
    
    try:
        print(f" - First hotel is {first_result['name']}")
        hotel_name.append(first_result['name'])
        hotel_lat.append(first_result['geometry']['location']['lat'])
        hotel_lng.append(first_result['geometry']['location']['lng'])

    except (KeyError, IndexError):
        print("- Missing field/result... skipping.")
        
## Print Log Footer
print("------------------------------")        
print("Data Retrieval Complete")
print("------------------------------")

Beginning Data Retrieval
------------------------------
Processing Samandağ:
 - First hotel is Anadolu Palace Hotel
Processing Yining:
 - First hotel is Xiushui Hotel
Processing Giyani:
 - First hotel is Kulani Country Lodge
Processing Qurayyat:
 - First hotel is Mozon Furnished Aprtments
Processing Jingdezhen:
 - First hotel is Garden Hotel
Processing Broken Hill:
 - First hotel is Royal Exchange Hotel
Processing Antakya:
 - First hotel is The Liwan Hotel
Processing Yicheng:
 - First hotel is GreenTree Alliance Wuxi Yixing Jinyizhong Road Chengbei RT-Market Hotel
------------------------------
Data Retrieval Complete
------------------------------


In [14]:
hotel_df = ideal_cities_df[['City', 'Country']]
hotel_df.insert(0, 'Hotel Name', hotel_name)
hotel_df.insert(len(hotel_df.columns), 'Lat', hotel_lat)
hotel_df.insert(len(hotel_df.columns), 'Lng', hotel_lng)

hotel_df

,Hotel Name,City,Country,Lat,Lng
0,Anadolu Palace Hotel,Samandağ,TR,36.081699,35.947641
1,Xiushui Hotel,Yining,CN,29.031062,114.573041
2,Kulani Country Lodge,Giyani,ZA,-23.308243,30.685959
3,Mozon Furnished Aprtments,Qurayyat,SA,31.339827,37.327837
4,Garden Hotel,Jingdezhen,CN,29.288565,117.194128
5,Royal Exchange Hotel,Broken Hill,AU,-31.957779,141.465727
6,The Liwan Hotel,Antakya,TR,36.198883,36.160747
7,GreenTree Alliance Wuxi Yixing Jinyizhong Road...,Yicheng,CN,31.365135,119.839492


In [15]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))